In [2]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python3 (found version "3.8.2") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so
-- pybind11 v2.5.dev1
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/lisandro/Documents/mt/tp3
-- Configuring done
-- Generating done
-- Build files hav

In [3]:
import metnum as mt

import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import util

In [4]:
df = pd.read_csv('../data/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

### Creo lista de ciudades únicas que aparecen en el data set

In [5]:
%%time 
#Encontrar las ciudades que mejor explican su precio en base a los features que estan abajo

df = pd.read_csv('../data/train.csv')

lista_ciudades = df['ciudad'].unique().tolist()

CPU times: user 3.21 s, sys: 256 ms, total: 3.47 s
Wall time: 3.53 s


### Me quedo con las columnas que me interesan del data set para hacer el análisis: 
'antiguedad','habitaciones','garages','banos','gimnasio','usosmultiples','centroscomercialescercanos','precio'

In [20]:
%%time

df = pd.read_csv('../data/train.csv')

###Selecciono los atributos a considerar 

atributos = ['banos'] #No incluyo el precio

atributos_con_precio = ['banos','precio']

df1 = df[atributos_con_precio].dropna()

### Porcentaje de datos usados para entrenar / testar

limit = int(0.85 * len(df1))

### Convierto en array de numpy y entreno la regresión lineal con el porcentaje seleccionado de datos

x = df1[atributos].values
x.reshape(x.shape[1],x.shape[0])

y = df1[['precio']].values
y.reshape(y.shape[1], y.shape[0])

x_train, y_train = x[0:limit], y[0:limit]
x_test, y_test = x[limit:], y[limit:]

linreg = mt.LinearRegression()
linreg.fit(x_train, y_train)

### predicción y calculo de métricas

pred = linreg.predict(x_test)

errores = util.get_errors(pred, y_test)

213779
CPU times: user 3.07 s, sys: 75.5 ms, total: 3.14 s
Wall time: 3.15 s


In [ ]:

ciudad = lista_ciudades[4]

df1 = df[df['ciudad'] == ciudad]

df1 = df1[['antiguedad','habitaciones','garages','banos','gimnasio','usosmultiples','centroscomercialescercanos','precio']].dropna()

df1 = df1[df1['precio'] > 10000000]

    
